In [21]:
########################################################################################
######################### QUEL SCREEN TWITTER EST A ANALYSER ? #########################
########################################################################################
MonScreenName = 'realDonaldTrump'
Langue = 'english'

In [22]:
# IMPORT PANDAS
import pandas as pd
# IMPORT TWEEPY
import tweepy
# IMPORT JSON RE & STRING ...
import json
import re
import string
import operator 
from collections import Counter
# IMPORT EMOJI & EMOTICON'S
import emoji
# IMPORT NLTK
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# Gestion configparser
import configparser

In [23]:
# Gestion de la configuration
config = configparser.ConfigParser()
config.read('../Configuration.conf')

['../Configuration.conf']

In [24]:
# Obtention de mes identifiants Twitter
ma_consumer_key = config.get('Twitter', 'consumer_key')
ma_consumer_secret = config.get('Twitter', 'consumer_secret')
ma_Oauth_token = config.get('Twitter', 'Oauth_token')
ma_Oauth_token_secret = config.get('Twitter', 'Oauth_token_secret')

In [25]:
# IDENTIFICATION DE MON API TWITTER
auth = tweepy.OAuthHandler(ma_consumer_key, ma_consumer_secret)
auth.set_access_token(ma_Oauth_token, ma_Oauth_token_secret)
api = tweepy.API(auth)

In [26]:
# FONCTION OBTENTION TWEETS DANS DATAFRAME
def MaRechercheTweets(ScreenName):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = ScreenName, count = 200)
    #save most recent tweets
    alltweets.extend(new_tweets)
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = ScreenName, count = 200, max_id = oldest)
        #save most recent tweets
        alltweets.extend(new_tweets)
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        print ('... %s Tweets chargés' % (len(alltweets)))
    #transform the tweepy tweets into a 2D array
    #outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text] for tweet in alltweets]
    return outtweets

In [27]:
# FONCTION DE TRAITEMENT DU TEXTE DES TWEETS
def MonTraitementTweets(MonTweet, lowercase=False):
    Tokens = tokenize(MonTweet) # FONCTION(S) Tokenize
    TokensEmojis = [Token for Token in ''.join(c for c in MonTweet if c in emoji.UNICODE_EMOJI)] # Emoji's
    TokensEmojisSens = [emoji.demojize(Token) for Token in TokensEmojis] # Emoji's (Sens)
    TokensURLs = [Token for Token in Tokens if Token.startswith('http')] # URL's
    TokensHastags = [Token for Token in Tokens if Token.startswith('#')] # Hastag's
    TokensMentions = [Token for Token in Tokens if Token.startswith('@')] # Mention's
    TokensRestants = [Token for Token in Tokens if Token not in TokensEmojis
                                                and Token not in TokensURLs
                                                and Token not in TokensHastags
                                                and Token not in TokensMentions]
    TokensRestants = [Token.lower() for Token in TokensRestants if Token not in Punctuation] # Punctuation 
    TokensRestants = [Token.lower() for Token in TokensRestants if Token not in StopWords] # Stopwords                
    return pd.Series([TokensRestants, TokensEmojis, TokensEmojisSens, TokensURLs, TokensHastags, TokensMentions])

In [28]:
# FONCTION TOKENIZE
def tokenize(s):
    return tokens_re.findall(s)

In [29]:
# GESTION DES EMOTICON's POUR TOKENIZE
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [30]:
# GESTION REGEX POUR TOKENIZE
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

In [31]:
# GESTION DE LA PONCTUATION
Punctuation = list(string.punctuation) + ['“', '…', '”']

In [32]:
# GESTION DES STOPWORDS
if Langue == 'english':
    StopWords = stopwords.words(Langue) + ['rt', 'via']             # Pour l'anglais
if Langue == 'french':
    StopWords = stopwords.words(Langue) + ['rt', 'via', 'les']      # Pour le français

In [33]:
TweetsCherches = MaRechercheTweets(MonScreenName)
TweetsCherches_df = pd.DataFrame(TweetsCherches, columns = ['Tweet_Id', 'Tweet_Date', 'Tweet_Texte'])

... 400 Tweets chargés
... 600 Tweets chargés
... 800 Tweets chargés
... 1000 Tweets chargés
... 1200 Tweets chargés
... 1400 Tweets chargés
... 1600 Tweets chargés
... 1799 Tweets chargés
... 1799 Tweets chargés


In [34]:
# TRAITEMENT DU TEXTE DES TWEETS
TweetsCherches_df[['Tokens', 'Emojis', 'SensEmojis', 'URLs', 'Hastags', 'Mentions']] = TweetsCherches_df.apply(lambda x: MonTraitementTweets(x['Tweet_Texte']), axis = 1)
TweetsCherches_df

,Tweet_Id,Tweet_Date,Tweet_Texte,Tokens,Emojis,SensEmojis,URLs,Hastags,Mentions
0,1121053578603397120,2019-04-24 14:09:35,I didn’t call Bob Costa of the Washington Post...,"[’, call, bob, costa, washington, post, called...",[],[],[],[],[]
1,1121049166615142400,2019-04-24 13:52:04,"....Congress has no time to legislate, they on...","[congress, time, legislate, want, continue, wi...",[],[],[https://t.co/SR99p1tb72],[],[]
2,1121048120312389634,2019-04-24 13:47:54,"No Collusion, No Obstruction - there has NEVER...","[collusion, obstruction, never, president, tra...",[],[],[https://t.co/v8HpsttkC5],[],[]
3,1121044792945926144,2019-04-24 13:34:41,Can anyone comprehend what a GREAT job Border ...,"[anyone, comprehend, great, job, border, patro...",[],[],[https://t.co/G13GJJCTeV],[],[]
4,1121025624632647682,2019-04-24 12:18:31,.....are there no “High Crimes and Misdemeanor...,"[high, crimes, misdemeanors, crimes, crimes, c...",[],[],[https://t.co/CfcpFiyym7],[],[]
5,1121023509029892096,2019-04-24 12:10:06,"The Mueller Report, despite being written by A...","[mueller, report, despite, written, angry, dem...",[],[],[https://t.co/P0Xq2h0Boq],[],[]
6,1121021160827887616,2019-04-24 12:00:46,Mexico’s Soldiers recently pulled guns on our ...,"[mexico, ’, soldiers, recently, pulled, guns, ...",[],[],[https://t.co/bpPOy93S7x],[],[]
7,1121012339552194560,2019-04-24 11:25:43,"A very big Caravan of over 20,000 people start...","[big, caravan, 20,000, people, started, mexico...",[],[],[https://t.co/xoErbSRTwA],[],[]
8,1121010431819173888,2019-04-24 11:18:08,The American people deserve to know who is in ...,"[american, people, deserve, know, country, yes...",[],[],[https://t.co/xrFCfBHsXj],[],[]
9,1121006942502182913,2019-04-24 11:04:17,“Former CIA analyst Larry Johnson accuses Unit...,"[former, cia, analyst, larry, johnson, accuses...",[],[],[https://t.co/ShzvQun68s],[],[]


In [35]:
# ANALYSE DE L'ENSEMBLE DES TWEETS (MOTS UTILES)
Comptage = Counter()
TsLesTokens = []
for i in range (0, len(TweetsCherches_df)):
    TsLesTokens.extend(TweetsCherches_df['Tokens'][i])
Comptage.update(TsLesTokens)
print('********** ANALYSE DES TWEETS (MOTS UTILES)')
print(Comptage.most_common(20))

********** ANALYSE DES TWEETS (MOTS UTILES)
[('’', 293), ('great', 216), ('president', 184), ('border', 174), ('democrats', 151), ('people', 128), ('trump', 127), ('amp', 107), ('wall', 99), ('today', 98), ('new', 95), ('thank', 80), ('news', 72), ('country', 72), ('collusion', 71), ('many', 69), ('security', 68), ('fake', 67), ('years', 66), ('never', 65)]


In [36]:
# ANALYSE DE L'ENSEMBLE DES MENTIONS (PERSONNES RE-TWEETEES)
Comptage = Counter()
TsLesTokens = []
for i in range (0, len(TweetsCherches_df)):
    TsLesTokens.extend(TweetsCherches_df['Mentions'][i])
Comptage.update(TsLesTokens)
print('********** ANALYSE DES MENTIONS (PERSONNES RE-TWEETEES)')
print(Comptage.most_common(20))

********** ANALYSE DES MENTIONS (PERSONNES RE-TWEETEES)
[('@realDonaldTrump', 84), ('@WhiteHouse', 60), ('@TomFitton', 26), ('@FoxNews', 25), ('@FLOTUS', 21), ('@JudicialWatch', 19), ('@GOPChairwoman', 18), ('@foxandfriends', 12), ('@RealDonaldTrump', 12), ('@DonaldJTrumpJr', 12), ('@POTUS', 10), ('@LouDobbs', 10), ('@IvankaTrump', 10), ('@charliekirk11', 10), ('@paulsperry_', 10), ('@Jim_Jordan', 9), ('@seanhannity', 9), ('@dbongino', 9), ('@VP', 8), ('@GOP', 7)]


In [37]:
# ANALYSE DE L'ENSEMBLE DES HASHTAGS (REFERENCES)
Comptage = Counter()
TsLesTokens = []
for i in range (0, len(TweetsCherches_df)):
    TsLesTokens.extend(TweetsCherches_df['Hastags'][i])
Comptage.update(TsLesTokens)
print('********** ANALYSE DES HASHTAGS (REFERENCES)')
print(Comptage.most_common(20))

********** ANALYSE DES HASHTAGS (REFERENCES)
[('#MAGA', 10), ('#SOTU', 5), ('#Spygate', 2), ('#WGDP', 2), ('#', 2), ('#AmericaFirst', 2), ('#SchumerShutdown', 2), ('#FirstStepAct', 2), ('#ArmyNavyGame', 2), ('#G20Summit', 2), ('#TaxReform', 1), ('#OpportunityZones', 1), ("#Mueller's", 1), ('#MichaelCohen', 1), ('#DrainTheSwamp', 1), ('#EXCLUSIVE', 1), ('#BreakingNews', 1), ('#Trump', 1), ('#Newsmakers', 1), ('#EddieGallagher', 1)]


In [38]:
# ANALYSE DE L'ENSEMBLE DES EMOJIS (LEUR SENS)
Comptage = Counter()
TsLesTokens = []
for i in range (0, len(TweetsCherches_df)):
    TsLesTokens.extend(TweetsCherches_df['SensEmojis'][i])
Comptage.update(TsLesTokens)
print('********** ANALYSE DES EMOJIS (LEUR SENS)')
print(Comptage.most_common(20))

********** ANALYSE DES EMOJIS (LEUR SENS)
[(':white_heavy_check_mark:', 5), (':red_heart:', 4), (':police_car_light:', 2), (':shamrock:', 2), (':movie_camera:', 1), (':backhand_index_pointing_down:', 1), (':money_bag:', 1), (':backhand_index_pointing_right:', 1)]
